<a href="https://colab.research.google.com/github/muhammadanas2365/JWAS.jl/blob/master/Genome_wide_association_studies_using_JWAS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application of Bayesian multiple regression methods to GWAS

Bayesian multiple regression methods were developed to simultaneously fit all genotyped markers to phenotypes, allowing for **different assumptions of genetic architecture** (distribution of marker effects) of traits. These methods provide a flexible and reliable framework for genome-wide association studies (GWAS), especially the **Bayesian variable selection (BVS) models** with mixture prior, such as the BayesC and BayesB that implemented in **JWAS**.

In [2]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia CSV DataFrames Random Statistics Distributions LinearAlgebra SparseArrays Printf JWAS"  # CSV DataFrames Random Statistics JWAS Distributions LinearAlgebra
JULIA_PACKAGES_IF_GPU=""
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.7.1 on the current Colab Runtime...
2022-08-02 16:54:13 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.7/julia-1.7.1-linux-x86_64.tar.gz [123374573/123374573] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...


CalledProcessError: ignored

In [1]:
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets;
phenofile  = dataset("phenotypes.csv")
genofile   = dataset("genotypes.csv")
phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstrings=["NA"]);

SyntaxError: ignored

## Bayesian Alphabet Methods

* **BayesA** 

Each marker has a normal distribution with its own variance: $\alpha_i \sim N(0,\sigma^2_i)$

* **BayesB**

BayesA with a proportion of genetic markers having zero effects: $\alpha_i \sim \left\{
  \begin{array}\\
    N(0,\sigma^2_i) & \text{probability}\ (1-\pi) \\
    0 & \text{probability}\ (\pi)
  \end{array} \right. $
  
* **BayesC**

A proportion of genetic markers having zero effects, and the others follow a normal distribution with a common variance: $\alpha_i \sim \left\{
  \begin{array}\\
    N(0,\sigma^2_\alpha) & \text{probability}\ (1-\pi) \\
    0 & \text{probability}\ (\pi)
  \end{array} \right. $
  
* **BayesC0 (equivalent to GBLUP)**

Each marker has a normal distribution with a common variance: $\alpha_i \sim N(0,\sigma_\alpha^2)$




In [ ]:
genotypes  = get_genotypes(genofile,separator=',',method="BayesC");

---

## Inference of single marker and group of markers (genomic window)

* **Inference of single marker**: posterior inclusion probability (referred to as model frequency in JWAS) of single markers

$$PIP_i = \frac{\sum_{n_t} \mathbf{1}_{\alpha^{(t)}_i \neq 0}}{n_t}$$

   **Single Trait Analysis**

In [ ]:
# Step 3: Build Model Equations
model_equation  ="y1 = intercept + x1 + x2 + genotypes";
model = build_model(model_equation);
# Step 4: Set Factors or Covariates
set_covariate(model,"x1");
# Step 5: Set Random or Fixed Effects
set_random(model,"x2");
out=runMCMC(model,phenotypes,chain_length = 100);

In [ ]:
marker_effects_file="results/MCMC_samples_marker_effects_genotypes_y1.txt"
GWAS(marker_effects_file,header=true)

 **Multiple Trait Analysis**

In [ ]:
pedfile    = dataset("pedigree.csv")
pedigree   = get_pedigree(pedfile,separator=",",header=true);
genotypes  = get_genotypes(genofile,separator=',',method="BayesC",quality_control = false);
# Step 3: Build Model Equations
model_equation  ="y1 = intercept + x1 + x2 + genotypes
                  y2 = intercept + x1 + x2 + x2*x3 + ID + dam + genotypes";
model = build_model(model_equation);
set_covariate(model,"x1");
set_random(model,"x2");
set_random(model,"ID dam",pedigree);
out=runMCMC(model,phenotypes);

In [ ]:
marker_effects_file="results1/MCMC_samples_marker_effects_genotypes_y1.txt"
GWAS(marker_effects_file,header=true)

*  **Inference of genomic window**: window posterior probability of assciation (WPPA) of genomic windows
   * *When causal loci are not included in the genotype data (e.g., SNP panel), the signal from a causative locus can be captured jointly by a group of SNPs that are in LD with the causal locus. Thus, a window of SNPs around the causative locus can better capture association signals than an individual SNP.*
   
WPPA was inferred based on the posterior probability that the proportion of the genetic variance explained by the markers in a genomic window is larger than a pre-defined proportion of total genetic variance. That is, for a genomic window k, its local GEBV across n individuals is computed as 
$$\mathbf{a}_k = \mathbf{M}_k \boldsymbol{\alpha}_k$$

Then the variance explained by the window is computed as 

$$ \sigma^2_{a_k} = var(\mathbf{a}_k) $$

The total genetic variance explained by all markers is computed as 

$$ \sigma^2_{a} = var(\mathbf{M} \boldsymbol{\alpha}) $$

Hence, the proportion of genetic variance that is explained by markers in window k is defined as

$$q_k = \frac{\sigma^2_{a_k}}{\sigma^2_{a}}$$

Suppose that we deem genomic windows that explain $0.1\%$ of the total genetic variance as potential windows containing causal variants, 

$$\text{WPPA}_k = \frac{\sum_{n_t} \mathbf{1}_{q_k^{(t)} \geq 0.001}}{n_t}$$


Classical multiple-testing corrections such as Bonferroni in frequentist-based single-SNP GWA methods do not account for dependencies of genotypes at SNPs that are in LD with each other, leading to very stringent significant threshold and low inference power. 
Instead, using simulation, Fernando et al. (2017) showed that the value of WPPA are able to control the proportion of false positives among the positive results, which is independent of the number of tests or correlations among them. That is, for posterior type I error rate (PER), we have 

$$ \begin{align*}PER & = Pr(H_0 \text{ is true}|\text{WPPA} > t)\\ & < 1-t \end{align*}$$

In [ ]:
map_file=dataset("map.csv")
first(CSV.read(map_file,DataFrame),6)

In [ ]:
marker_effects_file="results1/MCMC_samples_marker_effects_genotypes_y1.txt"
out=GWAS(model,map_file,marker_effects_file,header=true,window_size = "1 Mb", threshold = 0.01)
print(out)

---

# Using data on both genotyped and ungenotyped animals for GWA 

Genotypes of ungenotyped individuals are imputed using pedigree-based regression methods, while imputation errors are modeled directly to account for uncertainty.

In [ ]:
genotypes  = get_genotypes(genofile,separator=',',method="BayesC");

# Step 3: Build Model Equations
model_equation  ="y1 = intercept + x1 + x2 + x2*x3 + ID + dam + genotypes";
model = build_model(model_equation);

# Step 4: Set Factors or Covariates
set_covariate(model,"x1");

# Step 5: Set Random or Fixed Effects
set_random(model,"x2");
set_random(model,"ID dam",pedigree);

# Step 6: Run Analysis
out=runMCMC(model,phenotypes,single_step_analysis=true,pedigree=pedigree);

---

#### Applied to multiple traits, Bayesian GWA analyses in JWAS are able to study pleiotropic effects by multi-trait Bayesian regression methods, or Bayesian structural equation models.

# Bayesian GWA models to detect pleiotropic QTL

In addition to analysis of individual traits, there is also an interest in understanding genetic correlations between traits. In JWAS, **correlations of window GEBV** in multi-trait analysis is able to serve as a tool to identify pleiotropic regions. 

In [ ]:
genotypes  = get_genotypes(genofile,separator=',',method="BayesC",quality_control = false);
# Step 3: Build Model Equations
model_equation  ="y1 = intercept + x1 + x2 + genotypes
                  y2 = intercept + x1 + x2 + x2*x3 + ID + dam + genotypes";
model = build_model(model_equation);
set_covariate(model,"x1");
set_random(model,"x2");
set_random(model,"ID dam",pedigree);
out=runMCMC(model,phenotypes);

In [ ]:
map_file=dataset("map.csv")
marker_effects_file1 ="results3/MCMC_samples_marker_effects_genotypes_y1.txt"
marker_effects_file2 ="results3/MCMC_samples_marker_effects_genotypes_y2.txt"
out=GWAS(model,map_file,marker_effects_file1,marker_effects_file2,genetic_correlation=true,header=true,window_size="1 Mb")

In [ ]:
show(out[1],allcols=true)

In [ ]:
show(out[2],allcols=true)

In [ ]:
show(out[3],allcols=true)

However, estimates of genetic correlations based on multiple-marker regression models can misrepresent the true genetic parameters if the causal loci are not genotyped. For example, it may be difficult to differentiate the presence of a pleiotropic QTL from the presence of two closely linked single-trait QTL, depending on the extent of LD in the region.

---

# GWA using Bayesian structural equations models

Structural equation models (SEM) aim at going beyond correlations to making inferences on causal relationships among traits. The SEM Bayesian variable selection models are implemented in GWAS to introduce causal relationships. It is shown that the SEM BVS models have similar power to detect QTL as multi-trait BVS, but provide greater insight into biological mechanisms of the effects of QTL on traits through direct and indirect effects.

In [ ]:
genotypes  = get_genotypes(genofile,separator=',',method="BayesC",quality_control = false);
# Step 3: Build Model Equations
model_equation  ="y1 = intercept + x1 + x2 + x2*x3 + ID + dam + genotypes
                  y2 = intercept + x1 + x2 + ID + genotypes
                  y3 = intercept + x1 + ID + genotypes";
model = build_model(model_equation);

# Step 4: Set Factors or Covariates
set_covariate(model,"x1");

# Step 5: Set Random or Fixed Effects
set_random(model,"x2");
set_random(model,"ID dam",pedigree);

# Step 6: Run Analysis
# If `causal_structure` is provided, 
#e.g., causal_structure = [0.0 0.0 0.0;1.0 0.0 0.0;1.0 0.0 0.0] (column index affects row index, and a lower triangular matrix is required)
# for example: trait 1 -> trait 2 and trait 1 -> trait 3, phenotypic causal networks will be incorporated using structure equation models.
my_structure = [0.0 0.0 0.0
                1.0 0.0 0.0
                1.0 0.0 0.0]
out=runMCMC(model,phenotypes,causal_structure=my_structure);

**GWAS of Direct Marker Effects on Trait y2**

In [ ]:
#Compute the model frequency for each marker.
marker_effects_file="results4/MCMC_samples_marker_effects_genotypes_y2.txt"
GWAS(marker_effects_file,header=true)

In [ ]:
#Compute the posterior probability of association of the genomic window that explains a large proportion of the total genetic variance
map_file=dataset("map.csv")
marker_effects_file="results4/MCMC_samples_marker_effects_genotypes_y2.txt"
out=GWAS(model,map_file,marker_effects_file,header=true,window_size="1 Mb")
print(out)

**GWAS of Indirect Marker Effects on Trait y2**

In [ ]:
#Compute the model frequency for each marker.
marker_effects_file="results4/MCMC_samples_indirect_marker_effects_genotypes_y2.txt"
GWAS(marker_effects_file,header=true)

In [ ]:
#Compute the posterior probability of association of the genomic window that explains a large proportion of the total genetic variance
map_file=dataset("map.csv")
marker_effects_file="results4/MCMC_samples_indirect_marker_effects_genotypes_y2.txt"
out=GWAS(model,map_file,marker_effects_file,header=true,window_size="1 Mb")
print(out)

# Reference 

Fernando, R., Toosi, A., Wolc, A. et al. Application of Whole-Genome Prediction Methods for Genome-Wide Association Studies: A Bayesian Approach. JABES 22, 172–193 (2017). https://doi.org/10.1007/s13253-017-0277-6

Wolc, A., Dekkers, J.C.M. Application of Bayesian genomic prediction methods to genome-wide association analyses. Genet Sel Evol 54, 31 (2022). https://doi.org/10.1186/s12711-022-00724-8

